In [1]:
!pip install nltk numpy openpyxl Pillow pyparsing scikit-learn scipy tqdm keras scipy tensorflow transformers tensorflow-text tf-models-official

     |████████████████████████████████| 2.1MB 3.7MB/s 
     |████████████████████████████████| 3.4MB 32.3MB/s 
     |████████████████████████████████| 1.1MB 49.4MB/s 
     |████████████████████████████████| 3.3MB 42.1MB/s 
     |████████████████████████████████| 901kB 32.4MB/s 
     |████████████████████████████████| 706kB 30.2MB/s 
     |████████████████████████████████| 174kB 43.7MB/s 
     |████████████████████████████████| 358kB 36.7MB/s 
     |████████████████████████████████| 1.2MB 48.2MB/s 
     |████████████████████████████████| 38.2MB 87kB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=1bca3aa4e7a0e15ba95784a3da922853d78b1b908c7ce30079756bfe9d40c866
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.w

In [2]:
import nltk
import re
from openpyxl import load_workbook
from sklearn import preprocessing
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import numpy as np
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras import layers
# K Fold validation with the model
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
from keras.utils import to_categorical
from transformers import BertTokenizer, TFBertModel, TFBertPreTrainedModel, TFBertMainLayer, TFBertForSequenceClassification


nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

In [4]:
contractions = [
    (r"won\'t", "will not"),
    (r"can\'t", "cannot"),
    (r"i\'m", "i am"),
    (r"ain\'t", "is not"),
    (r"(\w+)\'ll", "\g<1> will"),
    (r"(\w+)n\'t", "\g<1> not"),
    (r"(\w+)\'ve", "\g<1> have"),
    (r"(\w+)\'s", "\g<1> is"),
    (r"(\w+)\'re", "\g<1> are"),
    (r"(\w+)\'d", "\g<1> would")
]


class Expander(object):
    def __init__(self, patterns=contractions):
        self.patterns = [(re.compile(regex, re.IGNORECASE), repl) for (regex, repl) in patterns]

    def expand(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
        return s

In [5]:
ALL_CLASSES = [
    "content discussion", "greeting", "logistics", "assignment instructions", "instruction question",
    "assignment question", "general comment", "response", "incomplete/typo", "feedback",
    "emoticon/non-verbal", "discussion wrap-up", "outside material", "opening statement",
    "general question", "content question", "general discussion"
]
DATASET_PATH = "dataset.xlsx"

# Opens a single sheet for reading, returns a list of messages and a list of classes
def read_worksheet(filename, sheet_name, all_classes, label_encoder, no_columns):
    wb = load_workbook(filename, read_only=True)
    ws = wb[sheet_name]
    column_labels = next(ws.rows)
    X = []
    y = []
    for row in ws.rows:
        if row[0].value is None:
            break
        elif row[0].value.strip() == "Course":  # Skip the first line which only contains column titles
            continue
        new_entry = {}
        for i in range(no_columns):
            new_entry[column_labels[i].value.lower().replace(" ", "_")] = str(row[i].value)
        c_list = [new_entry["codepreliminary"].lower().strip()]
        if c_list[0] not in all_classes:
            c_list = new_entry["codepreliminary"].lower().strip().split("/")
        # If there are 2 classes listed in document add message twice (1 for each class)
        for c in c_list:
            new_entry["codepreliminary"] = label_encoder.transform([c])[0]
            X.append(new_entry["message"])
            y.append(label_encoder.transform([c])[0])
    wb.close()
    return X, y

le = preprocessing.LabelEncoder()
le.fit(ALL_CLASSES)

LabelEncoder()

In [6]:
def preprocess_data(dataset, classes):
    # List of emojis found in messages
    emojis = [":)", ":(", ":D", "👍"]
    ps = PorterStemmer()
    tkn = TweetTokenizer()  # Use tweet tokenizer to not split emojis as punctuation
    exp = Expander()  # Expands contractions such as I'm, he's into I am, he is
    new_dataset = []
    new_classes = []
    stopword_set = stopwords.words("english")
    for i, entry in enumerate(dataset):
        exp.expand(entry)
        new_entry = []
        # Remove capitalization, stopwords
        for token in tkn.tokenize(entry):
            new_token = ps.stem(token)  # Stem the token and convert to lowercase
            if new_token.startswith("http"):
                new_token = "url"  # Replace links with a url tag
            if any(emoji in new_token for emoji in emojis):
                new_token = "emoji"  # Replace emojis with a string
            if (new_token not in stopword_set or len(new_token) == 1) and new_token.isalnum():
                new_entry.append(new_token)
        if len(new_entry) > 0:
            new_dataset.append(" ".join(new_entry))
            new_classes.append(classes[i])

    return new_dataset, new_classes

In [7]:
X, y = read_worksheet(DATASET_PATH, "Discussion only data", ALL_CLASSES, le, 10)
X2, y2 = read_worksheet(DATASET_PATH, "CREW data", ALL_CLASSES, le, 11)
X.extend(X2)  # Join worksheets into a single dataset
y.extend(y2)

In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
max_length = 256
batch_size = 6

In [14]:
def map_data_to_dict(input_ids, attention_masks, token_type_ids, classes):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, classes

In [32]:
def encode_data(data, classes):
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  class_list = []
  for i, msg in enumerate(data):
    input = tokenizer.encode_plus(
              msg,
              add_special_tokens=True,
              max_length=max_length,
              pad_to_max_length=True,
              return_attention_mask=True
            )
    input_ids_list.append(input["input_ids"])
    token_type_ids_list.append(input["token_type_ids"])
    attention_mask_list.append(input["attention_mask"])
    class_list.append([classes[i]])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, class_list)).map(map_data_to_dict)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = encode_data(X_train, y_train).shuffle(5000).batch(batch_size)
test_dataset = encode_data(X_test, y_test).batch(batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [40]:
learning_rate = 3e-5
no_epochs = 3
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(ALL_CLASSES))
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
history = model.fit(train_dataset, epochs=no_epochs, validation_data=test_dataset)

Epoch 1/3
113/113 [==============================] - 2157s 19s/step - loss: 2.1094 - accuracy: 0.4368 - val_loss: 1.5146 - val_accuracy: 0.5680
Epoch 2/3
113/113 [==============================] - 2145s 19s/step - loss: 1.2465 - accuracy: 0.6572 - val_loss: 1.3559 - val_accuracy: 0.5680
Epoch 3/3
113/113 [==============================] - 2137s 19s/step - loss: 0.8078 - accuracy: 0.7896 - val_loss: 1.3189 - val_accuracy: 0.6331
